# Type Hints in Functions

## Types Are Defined by Supported Operations

In [ ]:
# may be numeric (int, complex, Fraction, numpy.int32, etc)
# but it may also be a sequence (str, tuple, list, array, etc), or any other type that implements
# or inherits a __mul__ method that accepts an int argument
def double(x):
    return x * 2

In [ ]:
# annotated double
from collections import abc

def double(x: abc.Sequence):
    return x * 2

In [ ]:
from birds import *

woody = Bird()
alert(woody)  #1

In [ ]:
alert_duck(woody)  #2

In [ ]:
alert_bird(woody)  #3

## Types Usable in Annotations

### The Any Type

In [ ]:
from typing import Any

# when a type checker sees an untyped function like this
def double(x):
    return x * 2

# it assumes this
def double(x: Any) -> Any:
    return x * 2

# Any is assumed to support every possible operation

#### Subtype-of versus consistent-with

In [ ]:
class T1:
    ...

class T2(T1):
    ...

def f1(p: T1) -> None:
    ...

o2 = T2()
f1(o2)
# The call of f1(o2) is an application of the Liskov Substitution Principle (LSP). It defines the
# _is subtype-of_ in terms of supported operations: if an object of type T2 subistitutes an object
# of type T1 and the program still behaves correctly, then T2 is subtype-of T1

In [ ]:
def f2(p: T2) -> None:
    ...

o1 = T1()
f2(o1)  # type error
# From the point of view of supported operations, this make perfect sense: as a subclass,
# T2 inherits and must support all operations that T1 does. So an instance of T2 can be used
# anywhere an instance of T1 is expected. But the reverse is not necessary true: T2 may
# implement additional methos, so an instance of T1 may not be used everywhere an instace
# of T2 is expected

In [ ]:
def f3(p: Any) -> None:
    ...

o0 = object()
o1 = T1()
o2 = T2()

f3(o0)  #
f3(o1)  # all OK: rule #2
f3(o2)  #

def f4():  # implicity return type: 'Any'
    ...

o4 = f4()  # inferred type: 'Any'

f1(o4)  #
f2(o4)  # all OK: rule #3
f3(o4)  #

### Optional and Union Types

In [ ]:
# using the Optional special type for None as Default
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    ...

# Better syntax for Optional and Union in Python 3.10
def show_count(count: int, plural: str | None) -> str:
    ...

In [ ]:
# an example of a function that takes a str, but may return a str or float
from typing import Union

def parse_token(token: str) -> Union[str, float]:
    try:
        return float(token)
    except ValueError:
        return token

### Generic Collections

In [ ]:
# tokenize with type hints for Python >= 3.9
def tokenize(text: str) -> list[str]:
    return text.upper().split()

### Tuple Types

In [ ]:
# Tuples as records
# use the tuple built-in and declare types of the fields within []

# coordinates.py with the geohash function
from geolib import geohash as gh  # type: ignore #1

PRECISION = 9

def geohash(lat_lon: tuple[float, float]) -> str:  #2
    return gh.encode(*lat_lon, PRECISION)

In [ ]:
# Tuples as records with named fields
# to annotate a tuple with many fields, or specific types the code uses in many places
# prefer using typing.NamedTuple
from typing import NamedTuple
from geolib import geohash as gh  # type: ignore

PRECISION = 9

class Coordinate(NamedTuple):
    lat: float
    lon: float

def geohash(lat_lon: Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [ ]:
# Tuples as immutable sequences
# to annotate tuples of unspecified length that are used as immutable lists, you must specify
# a single type, followed by a comma and ... (Python's ellipsis)
# e.g. tuples[int, ...] is a tuple with int items
# there is no way to specify fiedls of different types for tuples of arbitrary length
# stuff: tuple[Any, ...] has the same effect as stuff: tuple, meaning stuff is a tuple of
# unspecified length with objects of any type
from collections.abc import Sequence

def columnize(sequence: Sequence[str], num_columns: int = 0) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    
    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)

    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]


animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize(animals)
table

### Generic Mapping

In [ ]:
# charindex.py
import sys
import re
import unicodedata
from collections.abc import Iterator


RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

def tokenize(text: str) -> Iterator[str]:  #1
    """return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()

def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}  #2
    for char in (chr(i) for i in range(start, end)):
        if name := unicodedata.name(char, ''):  #3
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    
    return index

In [ ]:
index = name_index(32, 65)
index

In [ ]:
index['SIGN']

In [ ]:
index['DIGIT']

In [ ]:
index['DIGIT'], index['EIGHT']

In [ ]:
index['DIGIT'] & index['EIGHT']

### Abstract Base Classes

In [ ]:
from collections.abc import Mapping

# Using abc.Mapping allows the caller to provide an instace of dict, defaultDict, ChainMap, a
# UserDict subclass, or any other type that is a subtype-of Mapping
def name2hex(name: str, color_map: Mapping[str, int]) -> str:
    ...


# Now color_map must be a dict or one of its subtypes, such as defaultDict or OrderedDict.
# In particular, a subclass of collections.UserDict would not pass the type check for color_map.
def name2hex(name: str, color_map: dict[str, int]) -> str:
    ...


**The return of a function is always a concrete object, so the return type hint shoud be a concrete type.**

In [ ]:
def tokenize(text: str) -> list[str]:
    return text.upper().split()

### Iterable

In [1]:
# replace.py
from collections.abc import Iterable

FromTo = tuple[str, str]  #1

def zip_replace(text: str, changes: Iterable[FromTo]) -> str:  #2
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

In [2]:
l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'

### Parametrized Generics and TypeVar

In [ ]:
# sample.py
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T')

def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

In [ ]:
# mode_float.py: mode that operates on float and subtypes
from collections import Counter
from collections.abc import Iterable

def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

In [ ]:
# Improved signature, but to permissive
from collections.abc import Iterable
from typing import TypeVar

T = TypeVar('T')

# every iterable is consistent-with Iterable[T], including iterables of unhashable types that
# collections.Counter cannot handle
def mode(data: Iterable[T]) -> T:
    ...


In [ ]:
# Restricted TypeVar
from collections.abc import Iterable
from decimal import Decimal
from fractions import Fraction
from typing import TypeVar

NumberT = TypeVar('T', float, Decimal, Fraction)

def mode(data: Iterable[NumberT]) -> NumberT:
    ...

In [ ]:
# Bounded TypeVar

# mode_hashable.py: same as previous example, with a more flexible signature
from collections import Counter
from collections.abc import Iterable, Hashable
from typing import TypeVar

HashableT = TypeVar('HashableT', bound=Hashable)

def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

In [ ]:
# The AnyStr predefined type variable
from typing import TypeVar #, AnyStr

# defined as
AnyStr = TypeVar('AnyStr', bytes, str)

# AnyStr is used in many functions that accept either bytes or str, and return values of the given type

### Static Protocols

In [1]:
from top import top

print(top([4, 1, 5, 2, 6, 7, 3], 3))

l = 'mango pear apple kiwi banana'.split()
print(top(l, 3))

l2 = [(len(s), s) for s in l]
print(l2)

print(l2, 3)

[7, 6, 5]
['pear', 'mango', 'kiwi']
[(5, 'mango'), (4, 'pear'), (5, 'apple'), (4, 'kiwi'), (6, 'banana')]
[(5, 'mango'), (4, 'pear'), (5, 'apple'), (4, 'kiwi'), (6, 'banana')] 3


### Callable

In [2]:
# illustrating variance
from collections.abc import Callable

def update(  #1
        probe: Callable[[], float],  #2
        display: Callable[[float], None]  #3
    ) -> None:
    temperature = probe()
    # imagine lots of control code here
    display(temperature)


def prob_ok() -> int:  #4
    return 42

def display_wrong(temperature: int) -> None:  #5
    print(hex(temperature))


update(prob_ok, display_wrong)  #6

def display_ok(temperature: complex) -> None:  #7
    print(temperature)

update(prob_ok, display_ok)

0x2a
42


### NoReturn
This is a special type used only to annotate the return type of functions that never return. Usually to raise exceptions.
There are dozen of such functions in the standard library.
For example sys.exit() raises SystemExit to terminate the Python process.

## Annotating Positional Only and Variadic Parameters